In [ ]:
# Config notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import sys
from importlib import reload
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun#, HADec, BaseRADecFrame

import qubic

In [ ]:
from pyoperators import (
    Cartesian2SphericalOperator, Rotation3dOperator,
    Spherical2CartesianOperator, rule_manager)
from pyoperators.utils import deprecated, isscalarlike
from pysimulators import (
    CartesianEquatorial2GalacticOperator,
    CartesianEquatorial2HorizontalOperator,
    CartesianHorizontal2EquatorialOperator,
    CartesianGalactic2EquatorialOperator,
    SamplingHorizontal,
    SphericalEquatorial2GalacticOperator,
    SphericalGalactic2EquatorialOperator,
    SphericalEquatorial2HorizontalOperator,
    SphericalHorizontal2EquatorialOperator)


In [ ]:
Salta = EarthLocation(lat=-24.731358*u.deg, lon=-65.409535*u.deg, height=1152*u.m)
utcoffset = -3*u.hour  # Eastern Daylight Time
door_az = 115.
az_span = 50.
el_min = 30.
el_max = 50.

In [ ]:
days = ['2022-07-12']
day = '2022-07-12'

In [ ]:
start_obs_hour = '00:00:00'
date = Time(day+ ' 00:00:00')
start_obs_date = day +' '+start_obs_hour
delta_time = np.linspace(12,30, 1000)*u.hour
time0 = Time(start_obs_date)-utcoffset
alltimes = time0 + delta_time
local_time_hours = ((Time(start_obs_date) + delta_time).cxcsec - date.cxcsec)/3600
### Local coordinates
frame_Salta = AltAz(obstime=alltimes, location=Salta)
### Moon
moon_Salta = get_moon(alltimes)
moonaltazs_Salta = moon_Salta.transform_to(frame_Salta)  
### Moon
sun_Salta = get_sun(alltimes)
sunaltazs_Salta = sun_Salta.transform_to(frame_Salta)  
delta_el = 20
valid = (moonaltazs_Salta.alt.value < (el_max+delta_el)) & \
        (moonaltazs_Salta.alt.value > (el_min-delta_el)) & \
        (moonaltazs_Salta.az.value > 80) & \
        (moonaltazs_Salta.az.value < 145)
tstart = np.min(local_time_hours[valid])
tstop = np.max(local_time_hours[valid])
local_start = str(Time(start_obs_date)+tstart*u.hour)[:16]
local_stop = str(Time(start_obs_date)+tstop*u.hour)[:16]
UTC_start = str(Time(start_obs_date)-utcoffset+tstart*u.hour)[:16]
UTC_stop = str(Time(start_obs_date)-utcoffset+tstop*u.hour)[:16]

In [ ]:
d = qubic.qubicdict.qubicDict()
d.read_from_file('MoonObservation.dict')
d['nf_sub'] = 1
d['date_obs'] = str(Time(start_obs_date)-utcoffset+tstart*u.hour)
d['latitude'] = -24.731377    ### Salta Regional Noroeste
d['longitude'] = -65.409546   ### Salta Regional Noroeste
d['sampling'] = 1.
moon_ra_mean = np.mean(moon_Salta.ra[valid]/u.deg)
moon_dec_mean = np.mean(moon_Salta.dec[valid]/u.deg)
d['RA_center'] = moon_ra_mean #deg
d['DEC_center'] = moon_dec_mean #deg
d['duration'] = tstop-tstart # Hours
d['angspeed'] = 0.8 #deg/s
d['delta_az'] = az_span #deg
d['nsweeps_per_elevation'] = 1
d['angspeed_psi'] = 0. #deg/s
backforthdt = d['delta_az'] / d['angspeed'] * 2
print('Scan Duration: ',backforthdt)
d['dead_time'] = 0.
print('Dead Time = {}'.format(d['dead_time']))
print('Sampling strategy, sweep?: {}, rand? {}, repeat? {}'.format(d['sweeping_pointing'],
                                                                  d['random_pointing'],
                                                                  d['repeat_pointing']))


In [ ]:
el_min = 30.
el_max = 50.
n_elevations = int(d['duration']*3600/(backforthdt+d['dead_time']))+1
el_step = np.round((el_max - el_min) / n_elevations * 100) / 100
d['fix_azimuth'] = {'apply':True,'az':105.,
                     'el':40,'el_step':el_step, 'fix_hwp':True, 'fix_pitch':True}
print(d['fix_azimuth'])
print('call')
p = qubic.get_pointing(d)
print(p.elevation)

el_min_final = np.min(p.elevation)
el_max_final = np.max(p.elevation)

In [ ]:
#d['RA_center'], d['DEC_center'] = 60,45#qubic.gal2equ(60, -45)
s = qubic.QubicScene(d)
    
q = qubic.QubicInstrument(d)
#q = qubic.QubicMultibandInstrument(d)
atod = qubic.QubicAcquisition(q, p, s, d)


In [ ]:
d1 = d.copy()
d1['sweeping_pointing'] = False
d1['repeat_pointing'] = True
d1['npointings'] = 9000
p1 = qubic.get_pointing(d1)
s1 = qubic.QubicScene(d1)    
q1 = qubic.QubicInstrument(d1)
#q = qubic.QubicMultibandInstrument(d)
atod1 = qubic.QubicAcquisition(q1, p1, s1, d1)
cov1 = atod1.get_coverage()

In [ ]:
cov = atod.get_coverage()

In [ ]:
plt.figure(figsize = (12,10))
hp.mollview(cov, title = "sweep", sub = 121)
hp.mollview(cov1, title = "repeat", sub = 122)

Plot in pyplot projected manually

In [ ]:
plt.plot(cov1)
plt.plot(cov)

In [ ]:
# quiero ver qué coordenadas de salida tiene usa el mapa healix en cada estrategia de apuntamiento
# Get the indexes of healpy map of the coverage map of sweeping
covmask = cov > 0
covindex = np.arange(0,12*256**2)[covmask] 
angscov = np.radians(np.array(hp.pix2ang(d['nside'], covindex, lonlat = True))) #theta, phi
# Same for repeat strategy
covmask1 = cov1 > 0
covindex1 = np.arange(0,12*256**2)[covmask1] 
angscov1 = np.radians(np.array(hp.pix2ang(d['nside'], covindex1, lonlat = True))) #theta, phi

In [ ]:
plt.figure(figsize= (12,4))
plt.subplot(121)
# convert to 1st and 4th quadrant the coordinates
fourth = angscov1[0][:] > np.pi
angscov1[0][fourth] = angscov1[0][fourth]-2*np.pi 
plt.scatter(angscov[0][:], angscov[1][:], label = 'sweep')
plt.scatter(angscov1[0][:], angscov1[1][:], label = 'repeat')
plt.xlabel(r'$long$')
plt.ylabel(r'$lat$')
plt.legend()
plt.subplot(122)
plt.plot(cov, label = 'sweep')
plt.plot(cov1, label = 'repeat')
plt.xlabel('index hp')
plt.ylabel('coverage')
plt.legend()

In [ ]:
plt.figure(figsize = (12,8))
plt.subplot(111, projection = 'mollweide')
plt.plot(angscov[0][:], angscov[1][:], label = 'sweep')
plt.plot(angscov1[0][:], angscov1[1][:],label = 'repeat')
plt.legend()

**Possible bug in rotations from horizontal2instrument (if fix_az) and galactic2instrument** (module: instrument.py, method: get_projection_operator, lines: 1255-1258)

Should we translate the az, el positions in p to galactic coordinates? 

In [ ]:
#Look at the centers
print("d", d['RA_center'], d['DEC_center'])
print("d1", d1['RA_center'], d1['DEC_center'])

In [ ]:
np.mean(p.equatorial, axis = 0), np.mean(p1.equatorial, axis = 0)
plt.figure(figsize = (12,4))
plt.subplot(121)
plt.scatter(p.equatorial[:,0], p.equatorial[:,1], alpha = 0.3, color = 'brown')
plt.scatter(p1.equatorial[:,0], p1.equatorial[:,1], alpha = 0.3, color = 'cyan')
plt.subplot(122)
mask = p.galactic[:,0] > 180
mask1 = p1.galactic[:,0] > 180
plt.scatter(p.galactic[mask,0]-360, p.galactic[mask,1], alpha = 0.3, color = 'brown')
plt.scatter(p.galactic[~mask,0], p.galactic[~mask,1], alpha = 0.3, color = 'brown')
plt.scatter(p1.galactic[mask1,0]-360, p1.galactic[mask1,1], alpha = 0.3, color = 'cyan')
plt.scatter(p1.galactic[~mask1,0], p1.galactic[~mask1,1], alpha = 0.3, color = 'cyan')


Debug sweep method

In [ ]:
def local_sweep(
        center, duration, period, angspeed, delta_az, nsweeps_per_elevation,
        angspeed_psi, maxpsi, hwp_stepsize, date_obs=None, latitude=None, longitude=None, fix_azimuth=None, random_hwp=True):

    nsamples = int(np.ceil(duration * 3600 / period))
    out = qubic.QubicSampling(
        nsamples, date_obs=date_obs, period=period, latitude=latitude,
        longitude=longitude)
    racenter = center[0]
    deccenter = center[1]
    backforthdt = delta_az / angspeed * 2

    # compute the sweep number
    isweeps = np.floor(out.time / backforthdt).astype(int)

    # azimuth/elevation of the center of the field as a function of time

    if fix_azimuth['apply']:
        azcenter = out.time * 0 + fix_azimuth['az']
        elcenter = out.time * 0 + fix_azimuth['el']
        print(azcenter, elcenter)
        #rot = Rotation3dOperator("ZY'", center[0], 90 - center[1], degrees=True)
        #s2c = Spherical2CartesianOperator('zenith,azimuth', degrees=True)
        #rotation = c2s(e2h(rot(s2c)))
        #coords = rotation(np.asarray([theta.T, phi.T]).T)
        #p.azimuth = coords[..., 0]
        #p.elevation = coords[..., 1]
    else:
        azcenter, elcenter = equ2hor(racenter, deccenter, out.time, date_obs=out.date_obs, latitude=out.latitude,
                                     longitude=out.longitude)

    # compute azimuth offset for all time samples
    daz = out.time * angspeed
    daz = daz % (delta_az * 2)
    mask = daz > delta_az
    daz[mask] = -daz[mask] + 2 * delta_az
    daz -= delta_az / 2

    # elevation is kept constant during nsweeps_per_elevation
    elcst = np.zeros(nsamples)
    ielevations = isweeps // nsweeps_per_elevation
    nelevations = ielevations[-1] + 1
    for i in range(nelevations):
        mask = ielevations == i
        elcst[mask] = np.mean(elcenter[mask])
        if fix_azimuth is not None:
            if fix_azimuth['apply']:
                el_step = fix_azimuth['el_step']
                elcst[mask] = elcenter[mask] - nelevations / 2 * el_step + i * el_step

    # azimuth and elevations to use for pointing
    azptg = azcenter + daz
    elptg = elcst
    plt.scatter(azptg, elptg)

    ### scan psi as well
    pitch = out.time * angspeed_psi
    pitch = pitch % (4 * maxpsi)
    mask = pitch > (2 * maxpsi)
    pitch[mask] = -pitch[mask] + 4 * maxpsi
    pitch -= maxpsi

    out.azimuth = azptg
    out.elevation = elptg
    out.pitch = pitch
    if random_hwp:
        out.angle_hwp = np.random.randint(0, int(90 / hwp_stepsize + 1), nsamples) * hwp_stepsize
    else:
        out.angle_hwp = np.zeros(nsamples)
        max_sweeps = np.max(isweeps)
        delta = int(nsamples / max_sweeps)
        for i in range(max_sweeps):
            out.angle_hwp[i * delta:(i + 1) * delta] = hwp_stepsize * np.mod(i, int(90 / hwp_stepsize + 1))

    if fix_azimuth['apply']:
        out.fix_az = True
        if fix_azimuth['fix_hwp']:
            out.angle_hwp = out.pitch * 0 + hwp_stepsize
        if fix_azimuth['fix_pitch']:
            out.pitch = 0
    else:
        out.fix_az = False

    return out

In [ ]:
center = (d['RA_center'], d['DEC_center'])
local_p = local_sweep(center, d['duration'], d['period'],
             d['angspeed'], d['delta_az'],
             d['nsweeps_per_elevation'],
             d['angspeed_psi'], d['maxpsi'], d['hwp_stepsize'],
             date_obs=d['date_obs'],
             latitude=d['latitude'],
             longitude=d['longitude'],
             fix_azimuth=d['fix_azimuth'], random_hwp=d['random_hwp'])

In [ ]:
maparr = np.zeros((12*d['nside']**2))
maparr[hp.ang2pix(256, np.radians(90-local_p.elevation), np.radians(local_p.azimuth))] = 1
hp.mollview(maparr)

In Healpix space the coverage is computed in a horizontal reference frame. Not in equatorial or galactic one. We have to perform a rotation so all the pointings has the same output

In [ ]:
def local_sweep_rotated(
        center, duration, period, angspeed, delta_az, nsweeps_per_elevation,
        angspeed_psi, maxpsi, hwp_stepsize, date_obs=None, latitude=None, longitude=None, fix_azimuth=None, random_hwp=True):

    nsamples = int(np.ceil(duration * 3600 / period))
    out = qubic.QubicSampling(
        nsamples, date_obs=date_obs, period=period, latitude=latitude,
        longitude=longitude)
    racenter = center[0]
    deccenter = center[1]
    backforthdt = delta_az / angspeed * 2

    # compute the sweep number
    isweeps = np.floor(out.time / backforthdt).astype(int)

    # azimuth/elevation of the center of the field as a function of time

    if fix_azimuth['apply']:
        azcenter = out.time * 0 + fix_azimuth['az']
        elcenter = out.time * 0 + fix_azimuth['el']
        print(azcenter, elcenter)
    else:
        azcenter, elcenter = equ2hor(racenter, deccenter, out.time, date_obs=out.date_obs, latitude=out.latitude,
                                     longitude=out.longitude)

    # compute azimuth offset for all time samples
    daz = out.time * angspeed
    daz = daz % (delta_az * 2)
    mask = daz > delta_az
    daz[mask] = -daz[mask] + 2 * delta_az
    daz -= delta_az / 2

    # elevation is kept constant during nsweeps_per_elevation
    elcst = np.zeros(nsamples)
    ielevations = isweeps // nsweeps_per_elevation
    nelevations = ielevations[-1] + 1
    for i in range(nelevations):
        mask = ielevations == i
        elcst[mask] = np.mean(elcenter[mask])
        if fix_azimuth is not None:
            if fix_azimuth['apply']:
                el_step = fix_azimuth['el_step']
                elcst[mask] = elcenter[mask] - nelevations / 2 * el_step + i * el_step

    # azimuth and elevations to use for pointing
    azptg = azcenter + daz
    elptg = elcst
    plt.scatter(azptg, elptg)
    # Rotation
    rot = Rotation3dOperator("ZY'", azcenter[0], 90 - elcenter[0], degrees=True)
    s2c = Spherical2CartesianOperator('zenith,azimuth', degrees=True)
    rotation = rot(s2c)
    coords = rotation(np.asarray([np.radians(90-elptg).T, np.radians(azptg).T]).T)
    azptg = coords[..., 0]
    elptg = coords[..., 1]

    ### scan psi as well
    pitch = out.time * angspeed_psi
    pitch = pitch % (4 * maxpsi)
    mask = pitch > (2 * maxpsi)
    pitch[mask] = -pitch[mask] + 4 * maxpsi
    pitch -= maxpsi

    out.azimuth = azptg
    out.elevation = elptg
    out.pitch = pitch
    if random_hwp:
        out.angle_hwp = np.random.randint(0, int(90 / hwp_stepsize + 1), nsamples) * hwp_stepsize
    else:
        out.angle_hwp = np.zeros(nsamples)
        max_sweeps = np.max(isweeps)
        delta = int(nsamples / max_sweeps)
        for i in range(max_sweeps):
            out.angle_hwp[i * delta:(i + 1) * delta] = hwp_stepsize * np.mod(i, int(90 / hwp_stepsize + 1))

    if fix_azimuth['apply']:
        out.fix_az = True
        if fix_azimuth['fix_hwp']:
            out.angle_hwp = out.pitch * 0 + hwp_stepsize
        if fix_azimuth['fix_pitch']:
            out.pitch = 0
    else:
        out.fix_az = False

    return out

In [ ]:
local_sweep_rotated(center, d['duration'], d['period'],
             d['angspeed'], d['delta_az'],
             d['nsweeps_per_elevation'],
             d['angspeed_psi'], d['maxpsi'], d['hwp_stepsize'],
             date_obs=d['date_obs'],
             latitude=d['latitude'],
             longitude=d['longitude'],
             fix_azimuth=d['fix_azimuth'], random_hwp=d['random_hwp'])

In [ ]:
maparr_rot = np.zeros((12*d['nside']**2))
maparr_rot[hp.ang2pix(256, np.radians(90-local_p.elevation), np.radians(local_p.azimuth))] = 1
hp.mollview(maparr_rot)

See hit_map()...looks fine

In [ ]:
hp.mollview(atod1.get_hitmap(), )
hp.mollview(atod.get_hitmap())